<img src="https://static.wixstatic.com/media/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png/v1/fill/w_1458,h_246,al_c,usm_0.66_1.00_0.01/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png" width="100">


 # __Notebook voor het importeren van 3.* auditfiles__

 
 __Auteur:__ Melvin Folkers (Yellow Stacks B.V.)<br>
 __Versie:__ 04 juli 2018<br>
 __Doel:__ Het importeren van auditfiles 3.* <br>
 ***

__overzicht van paragrafen in deze notebook__
 > __1. Parameters instellen__<br>
 
 > > 1.0 installeren van de benodigde bibliotheken<br>
 > 1.1 Notebook settings<br>
 > 1.2 Importeren van bibliotheken<br>

  
 > __2. functies__<br>
 > __3. Auditfile tabellen importeren__<br>
 > __4. Auditfile preppen__<br>
 > __5. Auditfile exporteren__<br>
 

__introductie__

Het doel van deze notebook is om auditfiles van het type 3 om te zetten in een dataset van grootboekmutaties.<br> 
De grootboekmutaties worden voorzien van informatie van de relaties, btwcodes, grootboek,dagboek, -en bedrijfsinformatie. <br>
In het script worden er ook validatiechecks gedaan op de ingelezen regels. <br>
De mutaties worden vervolgens geexporteerd naar een excelbestand.

Het script is gemaakt om __1 auditfile in te lezen__, ter demonstratie van wat python kan betekenen voor collega's werkzaam in de financiele branche (te denken aan: audit / tax / controlling).<br>
Dit soort scripts kunnen doorontwikkeld worden opdat meer auditfiles tergelijk kunnen worden ingelezen.<br>
Zorg ervoor dat wanneer je het script wil draaien, dat je de auditfile in dezelfde map zet als deze notebook.



# 1. Parameters instellen

als je voor de eerste keer jupyter notebook gebruikt moeten er nog wat bibliotheken (packages) geinstalleerd worden. Door onderstaande code te runnen met de Install parameter op <font color="green">__True__</font>.<br>
Wanneer je dat hebt gedaan kan je de waarde weer op <font color="green">__False__</font> zetten, omdat je dit maar 1 keer hoeft te doen.

### 1.0 installeren van de benodigde bibliotheken

In [2]:
install = False

if install == False : None
else : 
    !pip install numpy
    !pip install matplotlib
    !pip install pandas
    !pip install xml.etree.ElementTree
    !pip install matplotlib

### 1.1 Notebook settings
__instellingen voor gebruik van deze jupyter notebook__<br>
deze instellingen zorgen ervoor dat de notebook het hele scherm vult en dat er grafieken getoond kunnen worden.

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### 1.2 Importeren van bibliotheken
deze instellingen zorgen ervoor dat de notebook het hele scherm vult en dat er grafieken getoond kunnen worden. 

In [4]:

import sys, os
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

from datetime import datetime

***
# 2. Functies
__functies voor importeren van diverse lagen in de auditfile__ <br><br>
De functies die hieronder zijn geschreven zijn gemaakt om de meerder lagen van de auditfile door te lezen.<br>
Functies moeten altijd bovenaan het script worden gezet, omdat je deze later gaat aanroepen.<br>
Het echte script begint dus pas in __hoofdstuk 3__. let op dat je het onderstaande script runt tot hoofdstuk 3.

### 2.1 Functies voor het parsen van data

In [5]:
def parse_info(root):
   
    recordcontent = dict()
    
    for child in root:
        columnname = child.tag.replace(ns,'')
        columnvalue = child.text
        
        if len(child) == 0:
            recordcontent[columnname] = columnvalue     
            
        else: continue
            
    return recordcontent  

In [6]:
def stamtabellen(root) :    
    
    currentrow = 0
    records = dict()

    for child in root:

        module = child.tag.replace(ns,'')
        recordcontent = dict()

        for subchild in child:

            if len(subchild) != 0:
                for subsubchild in subchild:
                    columnvalue = subsubchild.text
                    columnname = subsubchild.tag.replace(ns,'')
                    recordcontent[columnname] = columnvalue

                    if len(subsubchild) != 0: print('let op! nog een sublaag gevonden')
            else:
                columnvalue =subchild.text
                columnname = subchild.tag.replace(ns,'')
                recordcontent[columnname] = columnvalue

        records[currentrow] = recordcontent
        currentrow +=1

    df = pd.DataFrame(records).transpose()
    return df

In [7]:
def tags_in_module(modules):
    tag = dict()
    
    for submodule in modules:
        tagname = submodule.tag.replace(ns,'')
        tag[tagname] = tag.get(tagname, 0) + 1
    return tag

### 2.2 Functies voor het converteren van data

In [8]:
def accounttype(dataframe):

    conditions = [
        (dataframe['accTp'] == 'P'),
        (dataframe['accTp'] == 'B')]

    choices = ['Winst & verlies','Balans']

    dataframe['accounttype'] = np.select(conditions, choices, default= 'onbekende balanstype' + dataframe['accTp'] )


In [9]:
def journaltype(dataframe):

    conditions = [
        (dataframe['jrnTp'] == 'Z'),
        (dataframe['jrnTp'] == 'B'),
        (dataframe['jrnTp'] == 'P'),
        (dataframe['jrnTp'] == 'O'),
        (dataframe['jrnTp'] == 'C'),
        (dataframe['jrnTp'] == 'M'),
        (dataframe['jrnTp'] == 'Y'),
        (dataframe['jrnTp'] == 'S')]

    choices = ['memoriaal', 'bankboek' , 'inkoopboek' , 'open/sluit balans', 'kasboek', 'memoriaal', 'salaris', 'verkoopboek']

    dataframe['journaltype'] = np.select(conditions, choices, default= 'onbekend dagboek' )

In [10]:
def vat_amount(dataframe):

    vat_amount_raw = dataframe['vatAmnt'].astype(float)
    
    conditions = [
        (dataframe['vatAmntTp'] == 'C'),
        (dataframe['vatAmntTp'] == 'D')]

    choices = [-1,1]

    dataframe['vat_amount'] = np.select(conditions, choices, default= 1 ) * vat_amount_raw


In [11]:
def amount(dataframe):

    amount_raw = dataframe['amnt'].astype(float)
    
    conditions = [
        (dataframe['amntTp'] == 'C'),
        (dataframe['amntTp'] == 'D')]

    choices = [-1,1]

    dataframe['amount'] = np.select(conditions, choices, default= 1 ) * amount_raw


***
# 3. Importeren van de auditfile
_starten met importeren van de auditfile data_

### 3.1 lees de auditfile in met de xml parser

__lees het xml bestand in__<br>
wanneer er meer dan 1 audifile instaat wordt de eerste auditfile in de lijst gekozen.<br>


In [12]:
pakket = 'multivers'

In [13]:
files = {'multivers':'Multivers_V32_2017.xaf', 'twinfield':'Twinfield_V31_2016.xaf','exact':'ExactOnline_V32_2016.xaf','onbekend': 'XmlAuditfileFinancieel3.2_test.xml'}

file  = r'/Users/melvinfolkers/Documents/github/Analytics/A_GENERAL/auditfiles/' + files[pakket]


In [14]:
tree = ET.parse(file)
root = tree.getroot()
namespaces = {'xsd':"http://www.w3.org/2001/XMLSchema", 'xsi':"http://www.w3.org/2001/XMLSchema-instance" }


_stel namespace vast_

In [15]:

ns_raw =  root.tag.split('{')[1].split('}')[0]
ns = '{'+ ns_raw + '}'
namespaces['af'] = ns_raw


### 3.2 Maak onderscheid tussen header en company subledgers

_splitst de auditfile op in 3 blokken._
> - header
> - company
> - transactions

In [16]:
header    = root.find('af:header',namespaces)
company   = root.find('af:company', namespaces)
transactions = root.find('af:company/af:transactions', namespaces)

_leest de eerste laag van het xml niveau in. Dit kunnen we zijn als de metadata van de auditfile_

In [17]:

headerinfo = pd.DataFrame(parse_info(header), index = [0])
companyinfo = pd.DataFrame(parse_info(company), index = [0])
transactioninfo = pd.DataFrame(parse_info(transactions), index = [0])
af_info = pd.concat([headerinfo, companyinfo, transactioninfo], axis = 1)

af_info

,curCode,dateCreated,endDate,fiscalYear,softwareDesc,softwareVersion,startDate,companyIdent,companyName,taxRegIdent,taxRegistrationCountry,linesCount,totalCredit,totalDebit
0,EUR,2017-05-08,2017-12-31,2017,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32


### 3.3 Parsen van data van de stamtabellen

_aanmaken van de volgende subtabellen:_
> - periods <br>
> - customer suppliers <br>
> - vatcodes <br>
> - generalLedger <br>
> - basics<br>

In [18]:
periods = stamtabellen(company.findall('af:periods/af:period',namespaces))
custsup = stamtabellen(company.findall('af:customersSuppliers/af:customerSupplier',namespaces))
vatcode = stamtabellen(company.findall('af:vatCodes/af:vatCode',namespaces))
genledg  = stamtabellen(company.findall('af:generalLedger/af:ledgerAccount',namespaces))
basics  = stamtabellen(company.findall('af:generalLedger/af:basics',namespaces))

In [19]:
basics.head()
genledg.head()
vatcode.head()
periods.head()
custsup.head()

,basicDesc,basicID,basicType
0,OH tuinen op abonnement,U-INT-02,04


,accDesc,accID,accTp,leadReference
0,Bedrijfsgebouwen,0020,B,BMvaBegVvp
1,Bedrijfsterreinen,0025,B,BMvaBegVvp
2,Computers en software,0030,B,BMvaObeVvp
3,Machines en installaties,0035,B,BMvaObeVvp
4,Transportmiddelen,0040,B,BMvaTevVvp


,vatDesc,vatID,vatToClaimAccID,vatToPayAccID
0,None,0,1710,1750
1,Laag tarief,1,1710,1760
2,Hoog tarief,2,1710,1770


,endDatePeriod,endTimePeriod,periodDesc,periodNumber,startDatePeriod,startTimePeriod
0,2017-01-31,23:59:59+00:00,None,1,2017-01-01,00:00:00+00:00
1,2017-02-28,23:59:59+00:00,None,2,2017-02-01,00:00:00+00:00
2,2017-03-31,23:59:59+00:00,None,3,2017-03-01,00:00:00+00:00
3,2017-04-30,23:59:59+00:00,None,4,2017-04-01,00:00:00+00:00
4,2017-05-31,23:59:59+00:00,None,5,2017-05-01,00:00:00+00:00


,bankAccNr,bankIdCd,city,contact,country,custSupID,custSupName,eMail,fax,postalCode,streetname,taxRegistrationCountry,telephone,website
0,0343022443,DORDRECHT,DORDRECHT,de heer J.A.M. van Bekkerem,NaN,D1001,Van Bekkerem B.V.,info@bekkerem.nl,078-6230231,2369 WE,Lorenzkade 39,NL,078-6230230,www.bekkerem.nl
1,102028,LEIDEN,LEIDEN,Mevr. M. van der Laan,NaN,D1002,Fa. Brokma Blokhutten v/h Gebr. Brokma,info@brokma-transport.nl,071-8936925,2346 GF,Frederik Hendrikplantsoen 23,NL,071-8950501,www.brokma-transport.nl
2,0159202078,HILVERSUM,HILVERSUM,Hr. B.R. van Haaften sr,NaN,D1003,Van Haaften International B.V.,marketing@vanhaaften.com,055-2229346,5600 AJ,Postbus 123,NL,055-2225553,www.vanhaaften.com
3,0123456789,ARLINGTON,ARLINGTON,Mr. Dave McFarland,US,D1004,Garden Impressions Beverly,NaN,0932-45667555,22121,345 New Upshurstreet,NaN,0932-45667564,NaN
4,1643124111,KARLSRUHE,KARLSRUHE,Herr Jozef von Weitzecker,DE,D1005,Gartencenter Löwenbrau,NaN,0928-234987222,245583,Dortmunderstrasse 20-40,DE,0928-234987987,NaN


### 3.4 Parsen van data van de journals

In [20]:
journals = company.findall('af:transactions/af:journal', namespaces)
journal_df = pd.DataFrame()

for journal in journals:
    jrninfo = dict()
    
    for records in journal:
        if len(records) == 0:
            columnnames = records.tag.replace(ns,'')
            columnvalues = records.text
            jrninfo[columnnames] = columnvalues
    journal_df = journal_df.append(jrninfo, ignore_index = True)

In [21]:
journal_df

,desc,jrnID,jrnTp,offsetAccID,bankAccNr
0,Activadagboek,A,Z,2017,NaN
1,ABN AMRO BANK,AA,B,1200,614973465
2,Inkoopboek,I,P,7090,NaN
3,ING BANK,ING,B,1110,661262286
4,Inkooporders,IO,P,3090,NaN
5,Kasboek,K,C,1000,NaN
6,Memoriaalboek,M,M,2000,NaN
7,Memoriaal Projecten,MPR,M,2000,NaN
8,Verkoopboek,V,S,8090,NaN
9,Voorraad goederenontvangst,VGO,Z,2020,NaN


### 3.5 Parsen van de transactions in de journals
In de journals zitten meerdere transactions. we willen van alle journals alle transactions hebben:
> journal 1 <br>
> journal 2 <br>
> journal 3 <br>
>> transaction 1 <br>
>> transaction 2 <br>
>> transaction 3 <br>
>>> transactionline 1  <font color='green'>kolomnamen, kolomnwaardes</font> <br>
>>> transactionline 2  <font color='green'>kolomnamen, kolomnwaardes</font> <br>
>>> transactionline 3  <font color='green'>kolomnamen, kolomnwaardes</font> <br>


In [22]:
transactions_df = pd.DataFrame()

total_records = list()
record_dict = dict()

for journal in journals:

    
    for records in journal:
        
        if len(records) == 0:
            columnnames = records.tag.replace(ns,'')
            columnvalues = records.text
            record_dict[columnnames] = columnvalues

        else:
            for record in records:
                if len(record) == 0:
                    columnnames = record.tag.replace(ns,'')
                    columnvalues = record.text
                    record_dict[columnnames] = columnvalues

                else:

                    for subfields in record:
                        if len(subfields) == 0:
                            columnnames = subfields.tag.replace(ns,'')
                            columnvalues = subfields.text
                            record_dict[columnnames] = columnvalues

                        else: 

                            for subfields_1 in subfields:
                                if len(subfields_1) == 0:
                                    columnnames = subfields_1.tag.replace(ns,'')
                                    columnvalues = subfields_1.text
                                    record_dict[columnnames] = columnvalues
                                else : print('nog een sublaag!')

                    
                    total_records.append(record_dict.copy())

transactions_df = transactions_df.append(total_records, ignore_index = True)


In [23]:
# transactions_df.head()
# transactions_df.describe()
# transactions_df.dtypes

***
# 4. Auditfile preppen

### 4.1 Journals voorzien van dagboektype
_classificeer dagboektype van de journals dataframe_

In [24]:
journaltype(journal_df)
journal_df = journal_df.drop(['jrnTp'] , axis = 1)

### 4.2 accounttype toevoegen aan general ledger


In [25]:
accounttype(genledg)

### 4.3 Transacties het juiste dataformat geven
_aanmaken van nieuwe velden en aanpassen van datatypes_

In [26]:
tr = transactions_df

amount(transactions_df)
tr['effDate'] = pd.to_datetime(transactions_df['effDate'])
tr['trDt'] = pd.to_datetime(transactions_df['trDt'])

if 'vatAmnt' in tr.columns:
    vat_amount(transactions_df)
    tr = tr.drop(['amnt', 'amntTp', 'vatAmnt', 'vatAmntTp'], axis=1)
else:
    print('geen vat amount!')


_verwijder kolommen die we niet meer nodig hebben:_

### 4.3 Tabellen aan elkaar koppelen
_De volgende stamtabellen worden aan elkaar gelinkt op unieke ID_
   1. periods
   2. vatcodes
   3. customers suppliers
   4. general ledger
   5. journalinfo
   6. auditfile info
   

In [27]:
print('length before joins:'.ljust(40), str(len(tr)).ljust(10), 'number of columns:', len(tr.columns), '\n')

# join met periods

temp_1 = pd.merge(tr, periods, on ='periodNumber', how='left')
print('length after join periods:'.ljust(40), str(len(temp_1)).ljust(10), 'number of columns:', len(temp_1.columns))

# join met vatcodes

temp_2 = pd.merge(temp_1, vatcode, on='vatID', how='left')
print('length after join vatcodes:'.ljust(40), str(len(temp_2)).ljust(10), 'number of columns:', len(temp_2.columns))

# join met customersuppliers

temp_3 = pd.merge(temp_2, custsup.add_prefix('cs_'), left_on='custSupID' , right_on = 'cs_custSupID', how='left')
print('length after join customersuppliers:'.ljust(40), str(len(temp_3)).ljust(10), 'number of columns:', len(temp_3.columns))

# join met generalLedger

temp_4 = pd.merge(temp_3, genledg, on='accID' , how='left')
print('length after join generalledger:'.ljust(40), str(len(temp_4)).ljust(10), 'number of columns:', len(temp_4.columns))


# join met journalinfo

temp_5= pd.merge(temp_4, journal_df.add_prefix('jrn_'),left_on = 'jrnID', right_on='jrn_jrnID' , how='left')
print('length after join journal:'.ljust(40), str(len(temp_5)).ljust(10), 'number of columns:', len(temp_5.columns))

# join met auditfile info

# temp_6= pd.merge(temp_5, af_info.add_prefix('info_'), on = None ,how='inner')
# print('length after join journal:'.ljust(40), str(len(temp_6)).ljust(10), 'number of columns:', len(temp_6.columns))


auditfile = temp_5


length before joins:                     371        number of columns: 22 

length after join periods:               371        number of columns: 27
length after join vatcodes:              371        number of columns: 30
length after join customersuppliers:     371        number of columns: 44
length after join generalledger:         371        number of columns: 48
length after join journal:               371        number of columns: 53


In [28]:
af_info

#af_info_formatted = 

,curCode,dateCreated,endDate,fiscalYear,softwareDesc,softwareVersion,startDate,companyIdent,companyName,taxRegIdent,taxRegistrationCountry,linesCount,totalCredit,totalDebit
0,EUR,2017-05-08,2017-12-31,2017,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32


In [29]:
af_info.iloc[0]["dateCreated"]

'2017-05-08'

In [30]:
columnnames = list(af_info)
number_of_columns = len(columnnames)
for column_name in columnnames:
    auditfile[column_name] = af_info.iloc[0][column_name]
    
    
auditfile.head()


,accID,bankAccNr,costID,curAmnt,curCode,custSupID,desc,docRef,effDate,invRef,...,softwareDesc,softwareVersion,startDate,companyIdent,companyName,taxRegIdent,taxRegistrationCountry,linesCount,totalCredit,totalDebit
0,4820,NaN,NaN,NaN,EUR,NaN,020 Bedrijfsgebouwen,1,2017-01-31,NaN,...,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32
1,0120,NaN,NaN,NaN,EUR,NaN,Automat. afschrijving per. 1,2,2017-01-31,NaN,...,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32
2,4830,NaN,NaN,NaN,EUR,NaN,030 Computers en software,1,2017-01-31,NaN,...,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32
3,0130,NaN,NaN,NaN,EUR,NaN,Automat. afschrijving per. 1,2,2017-01-31,NaN,...,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32
4,4835,NaN,NaN,NaN,EUR,NaN,035 Machines en installaties,1,2017-01-31,NaN,...,Unit4 Multivers,10.9.2658.33026,2017-01-01,MVL00003,Demonstratiebedrijf Homegarden,004532016,NL,371,135836.32,135836.32


***
# 5. Auditfile exporteren

In [31]:
writer = pd.ExcelWriter('auditfile.xlsx')
auditfile.to_excel(writer, 'export_af')
writer.save()